In [32]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
import pickle
import re
import nltk

with open("knock60.pkl", "rb") as f:
    model = pickle.load(f)

In [33]:
#国名取得：https://www.jpo.go.jp/system/patent/pct/tetuzuki/document/kuni-meisho/kuni-meisho-ichiran.pdf
df = pd.read_table("kuni-meisho-ichiran.csv",sep=",",encoding="shift-jis")
df = df.drop("Unnamed: 3",axis=1)#空白列が追加されちゃったので削除
df.head()

,日本語名称,英語名称,２文字コード
0,アイスランド共和国,Iceland,IS
1,アイルランド,Ireland,IE
2,アゼルバイジャン共和国,Azerbaijan,AZ
3,アフガニスタン,Afghanistan,AF
4,アメリカ合衆国,United States of America,US


In [34]:
def prep(row):#前処理をする．row:dataframe1行
    text = row["英語名称"]#英語名称だけを前処理対象にする
    t_prep = re.sub(r"-","",text)#ハイフン削除
    t_prep = re.sub(r'[\"\'.,:;\(\)#\|\*\+\!\?#$%&/\]\[\{\}_]', '', t_prep)#記号削除(主にハイフンを消したい)
    t_prep = re.sub(" ", "_", t_prep)#空白をアンダーバーに
    t_prep = t_prep.lower()#小文字化
    row["英語名称"] = t_prep#処理後の文字列に置き換え
    return row

df = df.apply(prep, axis=1)
country_english = df["英語名称"].tolist()
#ベクトル計算ができる国とその値だけ↓のリストに追加していく
vec_country=[]
vec_num=[]
df.head()

,日本語名称,英語名称,２文字コード
0,アイスランド共和国,iceland,IS
1,アイルランド,ireland,IE
2,アゼルバイジャン共和国,azerbaijan,AZ
3,アフガニスタン,afghanistan,AF
4,アメリカ合衆国,united_states_of_america,US


In [35]:
for country in country_english:#単語ベクトルを計算できるものはする
    try:
        vec = np.array(model.get_vector(country),"float64")
        vec_num.append(vec)
        vec_country.append(country)
    except:
        pass

In [36]:
pred = KMeans(n_clusters = 5,random_state=20030220).fit_predict(vec_num)#クラスター5つでクラスタリング
pred = list(pred)#0~4の5種類でクラスタリングされた

In [37]:
zipped = zip(pred,vec_country)#クラスタの数字と国名をくっつける
dic = {}
for content in zipped:
    dic.setdefault(content[0],[]).append(content[1])
    #setdefault:引数(キー，値)．[]キーがない時は追加する．国名をキーに対応して追加．

if __name__ == "__main__":
    for num in range (5):
        print(dic[num])

['afghanistan', 'israel', 'iraq', 'iran', 'haiti', 'pakistan', 'libya', 'lebanon']
['ireland', 'australia', 'canada', 'singapore', 'philippines', 'malaysia', 'south_africa']
['zimbabwe', 'sri_lanka', 'chad', 'bangladesh']
['iceland', 'yemen', 'uganda', 'ukraine', 'uruguay', 'ecuador', 'ethiopia', 'austria', 'oman', 'netherlands', 'ghana', 'qatar', 'guinea', 'cyprus', 'kuwait', 'croatia', 'kenya', 'costa_rica', 'colombia', 'congo', 'saudi_arabia', 'zambia', 'sierra_leone', 'jamaica', 'switzerland', 'sudan', 'somalia', 'china', 'chile', 'denmark', 'togo', 'dominican_republic', 'turkey', 'tonga', 'niger', 'nepal', 'norway', 'bahrain', 'panama', 'bahamas', 'hungary', 'fiji', 'finland', 'viet_nam', 'venezuela', 'peru', 'belgium', 'poland', 'mali', 'myanmar', 'monaco', 'morocco', 'romania']
['argentina', 'italy', 'india', 'indonesia', 'egypt', 'cuba', 'greece', 'georgia', 'sweden', 'spain', 'thailand', 'germany', 'nigeria', 'japan', 'brazil', 'france', 'portugal', 'malta', 'mexico', 'jordan'